# UNIT

- hugging-face: https://huggingface.co/yeeaa/UNIT_600M/tree/main



In [1]:
# !git clone https://github.com/yeezhu/UNIT.git

In [3]:
import torch
from PIL import Image
from transformers import CLIPImageProcessor

from unit import UNITModel

### uncomment to use Ascend NPU
# import torch_npu
# from torch_npu.npu import amp
# from torch_npu.contrib import transfer_to_npu

model_name = "yeeaa/UNIT_600M"

# use UNIT_600M model
# model_path = "/path/to/UNIT_600M/"
model_path = model_name

### uncomment to use UNIT_1B model
# model_path = "/path/to/UNIT_1B/"

# model = UNITModel.from_pretrained(model_path)
model = UNITModel.from_pretrained(model_name)

model.to(device="cuda")
model.eval()

image_processor = CLIPImageProcessor.from_pretrained(model_path)

In [ ]:
from metercls.libs.testor import ImageTestor


class Testor(ImageTestor):
  def preprocess_image(self, image):
    image_input = image_processor(image)["pixel_values"][0]
    image_tensor = torch.tensor(image_input).unsqueeze(0).to(torch.bfloat16).cuda()
    return image_tensor

  def predict(self, image_tensor):
    with torch.set_grad_enabled(False):
      cls_tokens, spatial_tokens = model(image_tensor)
    return cls_tokens, spatial_tokens
    ### Note: Applying a LayerNorm layer to these tokens is crucial before feeding them into LLMs.